In [1]:
from L420A import everything

In [2]:
everything.setup()

Biosphere database already present!!! No setup is needed
Database has already been imported


In [3]:
everything.constructor_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Construction')

In [4]:
everything.renovation_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Renovation')

In [5]:
mfa = everything.Mfa(2020, 2050)

In [6]:
for product in everything.Product.real_instances:
    mfa.add_points(product)

the EOL of the products that are not renovation possible is  concrete_foundation_production
the EOL of the products that are not renovation possible is  polystyrene foam slab production, 45% recycled
the EOL of the products that are not renovation possible is  polyethylene production, low density, granulate
the EOL of the products that are not renovation possible is  fibreboard production, hard
the EOL of the products that are not renovation possible is  steel production, low-alloyed, hot rolled
the EOL of the products that are not renovation possible is  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates
the EOL of the product is  wood wool production
the EOL of the products that are not renovation possible is  gypsum plasterboard production
the EOL of the products that are not renovation possible is  polyethylene production, high density, granulate, recycled
the EOL of the products that are not renovation possible is  p

In [7]:
mfa.points

[(2046, heat pump production),
 (2046, steel production, converter, low-alloyed),
 (2046, ventilation duct production, steel, 100x50 mm),
 (2046, polyethylene production, high density, granulate),
 (2046, hot water tank production, 600l),
 (2046, evacuated tube collector production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, gypsum plasterboard production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, market for plywood),
 (2050, fibre cement corrugated slab production),
 (2050, wood wool production),
 (2050, door production, outer, wood-glass),
 (2050, polyethylene production, high density, granulate, recycled),
 (2050, gypsum plasterboard production),
 (2050, wood wool product

In [ ]:
mfa

In [8]:
mfa.points

[(2046, heat pump production),
 (2046, steel production, converter, low-alloyed),
 (2046, ventilation duct production, steel, 100x50 mm),
 (2046, polyethylene production, high density, granulate),
 (2046, hot water tank production, 600l),
 (2046, evacuated tube collector production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, gypsum plasterboard production),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates),
 (2050, market for plywood),
 (2050, fibre cement corrugated slab production),
 (2050, wood wool production),
 (2050, door production, outer, wood-glass),
 (2050, polyethylene production, high density, granulate, recycled),
 (2050, gypsum plasterboard production),
 (2050, wood wool product

In [9]:
clean_list = mfa.clean_list_for_lca()

In [25]:
db = everything.bd.Database('ecoinvent-3.9.1-cuttoff')
db.search('treatment of waste concrete, inert material landfill')[2]['code']

'3d3a786bfd99f7e4e312c33c1def0b86'

In [20]:
clean_list
hr_clean_list = []
db = everything.bd.Database('ecoinvent-3.9.1-cuttoff')
for i in clean_list:
    # loop the list read the second element of each tuple, get the name of the activity with everything.bd.get_activity('code')
    print(i)
    activity = db.get(str(i[1]))
    print(activity)
    hr_clean_list.append( (i[0], activity['name']))
hr_clean_list

(2046, '1fc600ae9ced00237d14ddb9a46739fc')
'heat pump production, brine-water, 10kW' (unit, CH, None)
(2046, '2c559dbc4e822b444792e122dffe6c20')
'treatment of scrap steel, municipal incineration with fly ash extraction' (kilogram, CH, None)
(2046, '89cb4e1a47b707fe43b99135b81fcaba')
'steel production, converter, low-alloyed' (kilogram, RER, None)
(2046, '2c559dbc4e822b444792e122dffe6c20')
'treatment of scrap steel, municipal incineration with fly ash extraction' (kilogram, CH, None)
(2046, 'adc6bbccf3d5b2955b050510e647852a')
'ventilation duct production, steel, 100x50 mm' (meter, RER, None)
(2046, '2c559dbc4e822b444792e122dffe6c20')
'treatment of scrap steel, municipal incineration with fly ash extraction' (kilogram, CH, None)
(2046, '1580abc0d8542f9117775cc18e264f97')
'polyethylene production, high density, granulate' (kilogram, RER, None)
(2046, '2c559dbc4e822b444792e122dffe6c20')
'treatment of scrap steel, municipal incineration with fly ash extraction' (kilogram, CH, None)
(2046, '

ActivityDatasetDoesNotExist: <Model: ActivityDataset> instance matching query does not exist:
SQL: SELECT "t1"."id", "t1"."data", "t1"."code", "t1"."database", "t1"."location", "t1"."name", "t1"."product", "t1"."type" FROM "activitydataset" AS "t1" WHERE (("t1"."database" = ?) AND ("t1"."code" = ?)) ORDER BY Random() LIMIT ? OFFSET ?
Params: ['ecoinvent-3.9.1-cuttoff', '9526011b40bb76a51963028784dd315b', 1, 0]

In [9]:
lca2 = everything.ProLCA(activities_and_years=clean_list, yearly_databases=mfa.create_time_line_dict(), 
                         methods=[('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)'), ('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)')],
                         products=everything.Product.real_instances)

In [10]:
# renovation = lca2.give_me_embodied()

# # pickle this
# import pickle
# with open('renovation.pickle', 'wb') as f:
#     pickle.dump(renovation, f)

# get the pickle

import pickle

with open('renovation.pickle', 'rb') as f:
    renovation = pickle.load(f)

In [11]:
production = lca2.production_lca()

In [19]:
production

{(2020,
  concrete_foundation_production,
  'ecoinvent-3.9.1-cuttoff'): [13872.880638500286, 43.99707920471239],
 (2020,
  polystyrene foam slab production, 45% recycled,
  'ecoinvent-3.9.1-cuttoff'): [3419.038665225468, 12.786087097917745],
 (2020,
  polyethylene production, low density, granulate,
  'ecoinvent-3.9.1-cuttoff'): [319.209695386447, 1.1410022020801207],
 (2020,
  fibreboard production, hard,
  'ecoinvent-3.9.1-cuttoff'): [202.85713958731665, 4.3870538907674375],
 (2020,
  steel production, low-alloyed, hot rolled,
  'ecoinvent-3.9.1-cuttoff'): [1819.3230344217818, 7.737518979739819],
 (2020,
  concrete production, 25MPa, for building construction, for interior use, with cement ZN/D, with 50% RC-C aggregates,
  'ecoinvent-3.9.1-cuttoff'): [0.0, 0.0],
 (2020,
  glass wool mat production,
  'ecoinvent-3.9.1-cuttoff'): [2473.579532791892, 26.075566322188127],
 (2020,
  gypsum plasterboard production,
  'ecoinvent-3.9.1-cuttoff'): [494.72569017911144, 3.0273015047543606],
 (2

In [13]:
data = {}
for product in everything.Product.instances:
    product.embodied_impacts = everything.GiveMeName.aggregator(_11, product)
    if not len(product.embodied_impacts) == 0:
        data[product.name] = product.embodied_impacts

AttributeError: type object 'Product' has no attribute 'instances'

In [ ]:
from L420A import everything

In [ ]:
everything.setup()

In [ ]:
# data
# pickle this data

# Path: test.ipynb

import pickle
# with open('data.pickle', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data.pickle', 'rb') as handle:
    b = pickle.load(handle)
    

In [ ]:
(b)

In [ ]:
everything.GiveMeName.poop_to_excel(b, 'test.xlsx')

In [ ]:
db = everything.bd.Database('ecoinvent-3.9.1-cuttoff')

In [ ]:
for a in db:
    if a['code'] == '9fda2f5ef0c1309141b798483b169b23':
        print(a)

In [ ]:
db.search('treatment of scrap steel fly ash')[0]['code']